In [1]:
import numpy as np
from cluster_algorithms import base_kmeans
import matplotlib.pyplot as plt

from sklearn.model_selection import KFold
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import silhouette_score, davies_bouldin_score, calinski_harabasz_score
from scipy.spatial import Voronoi, voronoi_plot_2d
import time

In [2]:
data_files_path = '../data_files/data17_13TeV.AllPeriods.sgn.probes_lhmedium_EGAM2.bkg.VProbes_EGAM7.GRL_v97/'
file_name       = 'data17_13TeV.AllPeriods.sgn.probes_lhmedium_EGAM2.bkg.VProbes_EGAM7.GRL_v97_et0_eta0.npz'

plots_path      = '../clustering_plots/'
my_seed         = 13

In [3]:
def add_subplot_axes(ax,rect,axisbg='w'):
    fig = plt.gcf()
    box = ax.get_position()
    width = box.width
    height = box.height
    inax_position  = ax.transAxes.transform(rect[0:2])
    transFigure = fig.transFigure.inverted()
    infig_position = transFigure.transform(inax_position)    
    x = infig_position[0]
    y = infig_position[1]
    width *= rect[2]
    height *= rect[3]  # <= Typo was here
    subax = fig.add_axes([x,y,width,height],facecolor=axisbg)
    x_labelsize = subax.get_xticklabels()[0].get_size()
    y_labelsize = subax.get_yticklabels()[0].get_size()
    x_labelsize = rect[2]*0.5
    y_labelsize = rect[3]*0.5
    #subax.xaxis.set_tick_params(labelsize=x_labelsize)
    #subax.yaxis.set_tick_params(labelsize=y_labelsize)
    return subax

def plot_div_evo(al_object, breg_div, tag, path=plots_path):
    plt.figure(figsize=(10,8))    
    ax = plt.gca()
    ax.plot(range(al_object.get_last_iter()), al_object.get_sum_total_div(), '--o', c='g')
    ax.set_title('Total sum of the %s divergence' %(breg_div), fontsize=18)
    ax.set_ylabel(r'$D_{\phi}[C: D]$', fontsize=10)
    ax.set_xlabel(r'Iteractions', fontsize=10)
    ax.set_xticks(np.arange(1, al_object.get_last_iter()+ 1))
    plt.grid()
    ax2 = add_subplot_axes(ax, rect=[.3, .3, .6, .6])
    ax2.plot(range(al_object.get_last_iter()), al_object.get_sum_total_div(), '--o', c='g')
    ax2.set_ylabel(r'$D_{\phi}[C: D]$', fontsize=15)
    ax2.set_xlabel(r'Iteractions', fontsize=15)
    #ax2.set_xticks(np.arange(1, al_object.get_last_iter()+ 1))
    ax2.set_xlim([0, 8])
    ax2.grid()
    plt.savefig(path+'sum_total_divergence_ev_'+tag, dpi=100)
    plt.close()

def plot_voronoi2D_diagram(al_object, X, classes, divergence, tag, path=plots_path):
    
    centers = al_object.get_centroids()
    # Get the Voronoi diagrams
    vor = Voronoi(centers)
    ax_lim = [np.min(X, axis=0), np.max(X, axis=0)]
    fig, axes = plt.subplots(1, 1, figsize=(10,8))
    # Draw data using target to colorize them
    dict_label = {
        0 : ('red','Background'),
        1 : ('blue','Signal')
    }
    for i in np.unique(classes):
        axes.scatter(X[classes==i, 0], X[classes==i, 1], c=dict_label[i][0],
                     edgecolor='k', s=35, alpha=.5, label=dict_label[i][1])
    # Draw the centroids
    axes.plot(centers[:,0], centers[:,1], '^', c='black', markersize=15, label='Final Centroids')
    # Draw voronoi
    voronoi_plot_2d(vor, ax=axes, line_colors='darkorange', line_width=3, show_points=False, show_vertices=True)
    plt.title('Obtained Clusters for %s divergence' %(divergence), fontsize=18)
    plt.grid()
    plt.legend(loc='best', fontsize='x-large')
    plt.xlim([ax_lim[0][0], ax_lim[1][0]])
    plt.ylim([ax_lim[0][1], ax_lim[1][1]])
    plt.xticks(fontsize=13)
    plt.yticks(fontsize=13)
    plt.xlabel(r'$\langle\mu\rangle$', fontsize=15)
    plt.ylabel(r'$E_T$', fontsize=13)
    plt.savefig(path+'voronoi_diagram_'+tag, dpi=100)
    plt.close()

In [4]:
jpsi_data = dict(np.load(data_files_path+file_name))
jpsi_data.keys()

dict_keys(['features', 'etBins', 'etaBins', 'etBinIdx', 'etaBinIdx', 'data', 'target'])

In [5]:
list_of_features = list(jpsi_data['features'])
print(list_of_features)

['avgmu', 'L2Calo_ring_0', 'L2Calo_ring_1', 'L2Calo_ring_2', 'L2Calo_ring_3', 'L2Calo_ring_4', 'L2Calo_ring_5', 'L2Calo_ring_6', 'L2Calo_ring_7', 'L2Calo_ring_8', 'L2Calo_ring_9', 'L2Calo_ring_10', 'L2Calo_ring_11', 'L2Calo_ring_12', 'L2Calo_ring_13', 'L2Calo_ring_14', 'L2Calo_ring_15', 'L2Calo_ring_16', 'L2Calo_ring_17', 'L2Calo_ring_18', 'L2Calo_ring_19', 'L2Calo_ring_20', 'L2Calo_ring_21', 'L2Calo_ring_22', 'L2Calo_ring_23', 'L2Calo_ring_24', 'L2Calo_ring_25', 'L2Calo_ring_26', 'L2Calo_ring_27', 'L2Calo_ring_28', 'L2Calo_ring_29', 'L2Calo_ring_30', 'L2Calo_ring_31', 'L2Calo_ring_32', 'L2Calo_ring_33', 'L2Calo_ring_34', 'L2Calo_ring_35', 'L2Calo_ring_36', 'L2Calo_ring_37', 'L2Calo_ring_38', 'L2Calo_ring_39', 'L2Calo_ring_40', 'L2Calo_ring_41', 'L2Calo_ring_42', 'L2Calo_ring_43', 'L2Calo_ring_44', 'L2Calo_ring_45', 'L2Calo_ring_46', 'L2Calo_ring_47', 'L2Calo_ring_48', 'L2Calo_ring_49', 'L2Calo_ring_50', 'L2Calo_ring_51', 'L2Calo_ring_52', 'L2Calo_ring_53', 'L2Calo_ring_54', 'L2Calo_ri

In [6]:
var_indexes = [list_of_features.index('avgmu'),
               list_of_features.index('L2Calo_et'),]

In [7]:
data_      = jpsi_data['data'][:, var_indexes]
my_filter  = (data_[:,0] <= 80)
sgn_filter = jpsi_data['target'][my_filter]==1
bkg_filter = jpsi_data['target'][my_filter]==0
data_      = data_[my_filter,:]
y          = jpsi_data['target'][my_filter]
print(data_.shape)

(233397, 2)


In [8]:
sgn_choices_filter = np.random.choice(data_[sgn_filter].shape[0], size=800)
bkg_choices_filter = np.random.choice(data_[bkg_filter].shape[0], size=800)
choices_filter     = np.concatenate((sgn_choices_filter,bkg_choices_filter))

In [9]:
data_ = data_[choices_filter]
y     = jpsi_data['target'][choices_filter]
print(data_.shape)

(1600, 2)


In [10]:
GeV = 1e3
epsilon = 1e-1

In [11]:
data_[:, 1] = data_[:, 1]/GeV
#data_[data_[:,0] == 0, 0] = data_[data_[:,0] == 0, 0] + epsilon

In [12]:
n_clusters = [3, 4, 5]
n_folds    = 10
divs       = ['euclidean', 'exp', 'itakura-saito', 'gen_kl']#, 'gen_kls', 'gen_js']

In [13]:
cluster_measures = {
    'silhouette_score'        : silhouette_score,
    'davies_bouldin_score'    : davies_bouldin_score,
    'calinski_harabasz_score' : calinski_harabasz_score
}

In [14]:
kf = KFold(n_splits=n_folds, random_state=13)

In [15]:
CVO = list(kf.split(data_))

In [16]:
cv_dict = {}

In [17]:
for idiv in divs:
    cv_dict[idiv] = {}
    for idx, ifold in enumerate(CVO):
        trn_id, tst_id = ifold
        scaler         = MinMaxScaler(feature_range=(epsilon, 1))
        scaler.fit(data_[trn_id])
        norm_data      = scaler.transform(data_)
        cv_dict[idiv][idx] = {}
        for icluster in n_clusters:
            #print('Clustering with %i clusters using %s divergence in %i Fold...' %(icluster, idiv, idx))
            cv_dict[idiv][idx][icluster] = {}
            kmeans = base_kmeans(n_clusters=icluster)
            kmeans.fit(norm_data, n_iter=50, tol=1e-3, breg_div=idiv)
            plot_div_evo(kmeans, breg_div=idiv, tag='%s_%i_fold_%i_cluster' %(idiv, idx, icluster))
            plot_voronoi2D_diagram(kmeans, X=norm_data, classes=y, divergence=idiv,
                                   tag='%s_%i_fold_%i_cluster' %(idiv, idx, icluster))
            predicted_labels = kmeans.predict_cluster(norm_data[tst_id])
            for imeasure in cluster_measures.keys():
                cv_dict[idiv][idx][icluster][imeasure] = cluster_measures[imeasure](norm_data[tst_id],
                                                                                    predicted_labels)
            

The conversion criteria was reached... Stopping!
The conversion criteria was reached... Stopping!
The conversion criteria was reached... Stopping!
The conversion criteria was reached... Stopping!
The conversion criteria was reached... Stopping!
The conversion criteria was reached... Stopping!
The conversion criteria was reached... Stopping!
The conversion criteria was reached... Stopping!
The conversion criteria was reached... Stopping!
The conversion criteria was reached... Stopping!
The conversion criteria was reached... Stopping!
The conversion criteria was reached... Stopping!
The conversion criteria was reached... Stopping!
The conversion criteria was reached... Stopping!
The conversion criteria was reached... Stopping!
The conversion criteria was reached... Stopping!
The conversion criteria was reached... Stopping!
The conversion criteria was reached... Stopping!
The conversion criteria was reached... Stopping!
The conversion criteria was reached... Stopping!
The conversion crite

In [18]:
info_cluster_dict = {
    'bregman_divergence'      : [],
    'n_cluster'               : [],
    'silhouette_score'        : [],
    'davies_bouldin_score'    : [],
    'calinski_harabasz_score' : [],
}


In [19]:
for idiv in cv_dict.keys():
    for ifold in cv_dict[idiv].keys():
        for icluster in cv_dict[idiv][ifold].keys():
            info_cluster_dict['bregman_divergence'].append(idiv)
            info_cluster_dict['n_cluster'].append(icluster)
            for jmeasure in cluster_measures.keys():
                info_cluster_dict[jmeasure].append(cv_dict[idiv][ifold][icluster][jmeasure])

In [20]:
import pandas as pd

In [21]:
clus_df = pd.DataFrame(info_cluster_dict)

In [22]:
my_measure = list(cluster_measures.keys())

In [23]:
clus_df.head()

,bregman_divergence,n_cluster,silhouette_score,davies_bouldin_score,calinski_harabasz_score
0,euclidean,3,0.335935,0.944799,147.101728
1,euclidean,4,0.300294,1.126682,134.131398
2,euclidean,5,0.329511,0.885940,138.486358
3,euclidean,3,0.391564,0.950608,158.138321
4,euclidean,4,0.304507,1.169190,139.155629


In [24]:
cv_table = clus_df.groupby(['bregman_divergence', 'n_cluster'])[my_measure].agg(['mean', 'std'])

In [25]:
cv_table

silhouette_score           davies_bouldin_score  \
                                         mean       std                 mean   
bregman_divergence n_cluster                                                   
euclidean          3                 0.352291  0.026463             0.987560   
                   4                 0.313657  0.017839             1.011913   
                   5                 0.322835  0.019858             0.907166   
exp                3                 0.349739  0.027134             1.000536   
                   4                 0.321949  0.025290             0.986006   
                   5                 0.332213  0.015048             0.894387   
gen_kl             3                 0.354339  0.032294             1.007929   
                   4                 0.309102  0.029561             1.014581   
                   5                 0.311603  0.021565             0.973315   
itakura-saito      3                 0.342100  0.032462             1.076699   
                   4                 0.300809  0.029061             1.098582   
                   5                 0.301276  0.030773             1.025771   

                                       calinski_harabasz_score             
                                   std                    mean        std  
bregman_divergence n_cluster                                               
euclidean          3          0.090893              129.621501  20.290208  
                   4          0.101014              121.555708  16.822804  
                   5          0.044670              126.224371  13.608467  
exp                3          0.082581              129.299092  19.564928  
                   4          0.085164              120.128372  17.238963  
                   5          0.029217              125.607615  13.247866  
gen_kl             3          0.052846              127.958620  19.646539  
                   4          0.039595              118.375744  18.216333  
                   5          0.047848              119.324236  17.615023  
itakura-saito      3          0.083672              113.081859  23.006116  
                   4          0.119870              102.307193  15.757096  
                   5          0.026999              104.199398  16.120672

In [26]:
cv_table.round(2)

silhouette_score       davies_bouldin_score  \
                                         mean   std                 mean   
bregman_divergence n_cluster                                               
euclidean          3                     0.35  0.03                 0.99   
                   4                     0.31  0.02                 1.01   
                   5                     0.32  0.02                 0.91   
exp                3                     0.35  0.03                 1.00   
                   4                     0.32  0.03                 0.99   
                   5                     0.33  0.02                 0.89   
gen_kl             3                     0.35  0.03                 1.01   
                   4                     0.31  0.03                 1.01   
                   5                     0.31  0.02                 0.97   
itakura-saito      3                     0.34  0.03                 1.08   
                   4                     0.30  0.03                 1.10   
                   5                     0.30  0.03                 1.03   

                                   calinski_harabasz_score         
                               std                    mean    std  
bregman_divergence n_cluster                                       
euclidean          3          0.09                  129.62  20.29  
                   4          0.10                  121.56  16.82  
                   5          0.04                  126.22  13.61  
exp                3          0.08                  129.30  19.56  
                   4          0.09                  120.13  17.24  
                   5          0.03                  125.61  13.25  
gen_kl             3          0.05                  127.96  19.65  
                   4          0.04                  118.38  18.22  
                   5          0.05                  119.32  17.62  
itakura-saito      3          0.08                  113.08  23.01  
                   4          0.12                  102.31  15.76  
                   5          0.03                  104.20  16.12

* As melhores divergências foram a Euclidiana e a Exponencial;
* Itakura-saito obteve os piores resultados em todas os índices;

In [27]:
cv_table.round(2).to_excel('../data_files/clusterization_table.xlsx')

In [33]:
scaler         = MinMaxScaler(feature_range=(epsilon, 1))
norm_data      = scaler.fit_transform(data_)
icluster = 3
for idiv in divs:
    kmeans = base_kmeans(n_clusters=icluster)
    kmeans.fit(norm_data, n_iter=50, tol=1e-3, breg_div=idiv)
    plot_div_evo(kmeans, breg_div=idiv, tag='%s_%i_cluster_operation' %(idiv, icluster))
    plot_voronoi2D_diagram(kmeans, X=norm_data, classes=y, divergence=idiv,
                                       tag='%s_%i_cluster_operation' %(idiv, icluster))

The conversion criteria was reached... Stopping!
The conversion criteria was reached... Stopping!
The conversion criteria was reached... Stopping!
The conversion criteria was reached... Stopping!
